In [1]:
import git
import pandas as pd
import numpy as np
import scipy.integrate

import matplotlib.pyplot as plt

In [2]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [3]:
data = pd.read_csv(f"{homedir}/data/us/covid/deaths.csv")
data.columns

Index(['countyFIPS', 'County Name', 'State', 'stateFIPS', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20',
       '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20',
       '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20',
       '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20',
       '4/9/20'],
      dt

In [4]:
demo = pd.read_csv(f"{homedir}/data/us/demographics/county_populations.csv")
demo.columns

Index(['FIPS', 'total_pop', '60plus'], dtype='object')

In [5]:
data.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# countyFIPS is fucked up in the data for values 0... this fixes that shit by adding the stateFIPS to
# places with countyFITS = 0 and countyFIPS thus becomes unique
for i, check in enumerate(data['countyFIPS'] == 0):
    if data.at[i, 'stateFIPS'] == 0:
        if data.at[i, 'stateFIPS'] > 10:
            prefix = data.at[i, 'stateFIPS'] * 100
        else:
            prefix = data.at[i, 'stateFIPS'] * 1000
        data.at[i, 'countyFIPS'] = prefix + data.at[i, 'countyFIPS']

In [7]:
data.shape

(3196, 83)

In [8]:
# TODO fix data imputation
def q(t, N, shift):
    moving = np.array([57, 54, 52, 51, 49, 47, 46, 45, 44, 43, 39, 37, 34, 23, 19, 13, 10, 7, 6, 5, 5, 7, 6, 5, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 2])/100
    q = N*(1-moving) - shift*N
    if np.round(t) >= len(q):
        return q[-1]
    return q[int(np.round(t))]
    
def tau(t):
    return 0

def seiirq(dat, t, params, N, max_t, offset):
    if t >= max_t:
        return [0]*8
    beta = params[0]
    alpha = params[1] # rate from e to ia
    sigma = params[2] # rate of asymptomatic people becoming symptotic
    ra = params[3] # rate of asymptomatic recovery
    rs = params[4] # rate of symptomatic recovery
    delta = params[5] # death rate
    shift = params[6] # shift quarantine rate vertically from CityMapper data
    
    s = dat[0]
    e = dat[1]
    i_a = dat[2]
    i_s = dat[3]

    Qind = (q(t + offset, N, shift) - tau(t + offset)*i_a)/(s + e + i_a - tau(t + offset)*i_a)
    Qia = Qind + (1-Qind)*tau(t + offset)
    
    dsdt = - beta * s * i_a * (1 - Qind) * (1 - Qia) / N
    dedt = beta * s * i_a* (1 - Qind) * (1 - Qia) / N  - alpha * e
    diadt = alpha * e - (sigma + ra) * i_a
    disdt = sigma * i_a - (delta + rs) * i_s
    dddt = delta * i_s
    drdt = ra * i_a + rs * i_s
    
    
    # susceptible, exposed, infected, quarantined, recovered, died, unsusceptible
    out = [dsdt, dedt, diadt, disdt, drdt, dddt]

In [59]:
from sklearn.metrics import mean_squared_error

def mse(A, B):
    Ap = np.nan_to_num(A)
    Bp = np.nan_to_num(B)
    Ap[A == -np.inf] = 0
    Bp[B == -np.inf] = 0
    Ap[A == np.inf] = 0
    Bp[B == np.inf] = 0
    return mean_squared_error(Ap, Bp)

def model_z(params, data, pop, offset, tmax=-1):
    # initial conditions
    N = pop # total population
    initial_conditions = N * np.array(params[-4:]) # the parameters are a fraction of the population so multiply by the population
    
    e0 = initial_conditions[0]
    ia0 = initial_conditions[1]
    ids0 = initial_conditions[2]
    r0 = initial_conditions[3]
    
    d0 = data[0]
    s0 = N - np.sum(initial_conditions) - d0

    yz_0 = np.array([s0, e0, ia0, is0, r0, d0])
    
    n = data.shape[0]
    if tmax > 0:
        n = tmax
    
    # Package parameters into a tuple
    args = (params, N, n, offset)
    
    # Integrate ODEs
    try:
        s = scipy.integrate.odeint(seiirq, yz_0, np.arange(0, n), args=args)
    except RuntimeError:
#         print('RuntimeError', params)
        return np.zeros((n, len(yz_0)))

    return s

def fit_leastsq_z(params, data):
    Ddata = (data['Deaths'].values)
    Idata = (data['TotalCurrentlyPositive'].values)
    s = model_z(params, data)

    S = s[:,0]
    E = s[:,1]
    I_A = s[:,2]
    I_S = s[:,3]
    R = s[:,4]
    D = s[:,5]
    
    error = np.concatenate((D-Ddata, I_S - Idata))
    return error

In [51]:
# return data ever since first min_cases cases
def select_region(df, region, min_deaths=50):
    d = df.loc[df['countyFIPS'] == region]
    d = d[d.columns[4:]].values
    start = np.where(d > min_deaths)[1]
    infect = np.where(d > 0)[1]
    if start.size > 0:
         return (d[0][start[0]:], start[0] - infect[0])
    return (np.array([]), 0)

In [52]:
select_region(data, 6037)

(array([ 54.,  65.,  78.,  89., 117., 132., 147., 169., 198., 223.]), 20)

In [68]:
%matplotlib notebook
%matplotlib inline

plt.figure()
d, offset = select_region(data, 36047)
# parameters: beta, alpha, sigma, ra, rs, delta, shift
params = [1.8, 0.35, 0.1, 0.15, 0.34, 0.015, 0.5]
# conditions: E, IA, IS, R
initial_conditions = [4e-6, 0.0009, 0.0005, 0.0002]
s = model_z(params + initial_conditions, d, demo.loc[demo['FIPS'] == 36047]['total_pop'].values[0], offset)

<Figure size 432x288 with 0 Axes>

In [69]:
demo.loc[demo['FIPS'] == 6037]['total_pop'].values[0]

10105722

In [70]:
offset

11

In [71]:
d[0]

64.0

In [72]:
s

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [65]:
np.where(data['State'].values == "NY")

(array([1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871,
        1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882,
        1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893,
        1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904,
        1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915,
        1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924]),)

In [67]:
data[1861:1900]

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
1861,0,Statewide Unallocated,NY,36,0,0,0,0,0,0,...,232,249.0,418.0,559.0,423.0,276.0,305.0,305.0,240.0,240.0
1862,1,New York City Unallocated,NY,36,0,0,0,0,0,0,...,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1863,36001,Albany County,NY,36,0,0,0,0,0,0,...,1,2.0,2.0,4.0,6.0,8.0,9.0,9.0,9.0,12.0
1864,36003,Allegany County,NY,36,0,0,0,0,0,0,...,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1865,36005,Bronx County,NY,36,0,0,0,0,0,0,...,262,360.0,421.0,480.0,576.0,627.0,709.0,902.0,1001.0,1046.0
1866,36007,Broome County,NY,36,0,0,0,0,0,0,...,3,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,6.0
1867,36009,Cattaraugus County,NY,36,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1868,36011,Cayuga County,NY,36,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1869,36013,Chautauqua County,NY,36,0,0,0,0,0,0,...,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1870,36015,Chemung County,NY,36,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [73]:
d, offset = select_region(data, 36047)

In [74]:
diff = []
for n in diff 

array([  64.,   81.,  102.,  167.,  185.,  215.,  261.,  328.,  385.,
        485.,  610.,  896., 1022., 1153., 1313., 1473.])